In [ ]:
! pip install git+https://github.com/openai/whisper.git

In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
import whisper

from tqdm import tqdm

In [ ]:
model = whisper.load_model("base")

In [ ]:
# get texts
f = open("./darpa-timit/")
references = f.readlines()
references

In [ ]:
# get list of audio file names
for files in os.walk('./darpa-timit/'):
    recs = files[2]

recs

In [ ]:
path = "./darpa-timit/"

In [ ]:
# choose 100 random clips
import random

clips = []
for i in range(100):
    clips.append(random.choice(recs))

clips

In [ ]:
# get results
results = []
for clip in tqdm(clips):
    result = model.transcribe(path + clip, **dict(without_timestamps=True))["text"]
    results.append(result)

results

In [ ]:
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

data = pd.DataFrame(dict(transcription=results, reference=references))
data["transcription"] = [normalizer(text) for text in data["transcription"]]
data["reference"] = [normalizer(text) for text in data["reference"]]
data

In [ ]:
! pip install jiwer

In [ ]:
import jiwer

wer = jiwer.wer(list(data["reference"]), list(data["transcription"]))

print(f"WER: {wer * 100:.2f} %")